In [ ]:
import visvis as vv
import numpy as np
import sys
import os
import subprocess
import argparse
import matplotlib.pyplot as pl

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
fpath = '/Users/davidkohn/dev/obsidian/output/test_mhrw_synth_05/voxels/voxels0.npz'

In [ ]:
propIdx = 0

properties = ['Density',
              'LogSusceptibility',
              'ThermalConductivity',
              'ThermalProductivity',
              'LogResistivityX',
              'LogResistivityY',
              'LogResistivityZ',
              'ResistivityPhase',
              'PWaveVelocity']


In [ ]:
class Data(object):
    def __init__(self, inputFile):
        raw = np.load(inputFile)
        self.resolution = raw['resolution'].flatten()
        self.bounds = np.array([raw['x_bounds'], raw['y_bounds'], raw['z_bounds']])[:,:,0]
        self.nlayers = sum(1 for key in raw if key.startswith('layer'))
        self.nboundaries = sum(1 for key in raw if key.startswith('boundary'))
        self.properties = [raw[k] for k in properties]
        self.layers = [raw['layer'+str(k)]for k in range(self.nlayers)]
        self.boundaries = [raw['boundary'+str(k)]for k in range(self.nlayers)]
        self.samples = self.boundaries[0].shape[0]
        #reshape these key layers
        newshapeVox = (-1, self.resolution[2], self.resolution[1],self.resolution[0])
        newshapeSurf = (-1, self.resolution[1], self.resolution[0])
        self.layers = [k.reshape(newshapeVox, order='f') for k in self.layers]
        self.properties = [k.reshape(newshapeVox, order='f') for k in self.properties]
        self.boundaries = [k.reshape(newshapeSurf, order='f') for k in self.boundaries]
        # apply a 0/1 mask to each element of self.boundaries (which is a list of 3D arrays)
        """self.pbound = [
            np.sum([np.nonzero(sample) for sample in layer_sample_list]) / len(layer_sample_list) 
            for layer_sample_list in self.boundaries
        ]"""


In [ ]:
d = Data(fpath)

In [ ]:
prob_arr = 0
n_max = 10
layer = 0
for fpath_idx in range(0, n_max):
    print(fpath_idx)
    fpath = '/Users/davidkohn/dev/obsidian/output/gascoyne_config_01_01/voxels/voxels{}.npz'.format(fpath_idx)
    d = Data(fpath)
    X = np.linspace(d.bounds[0,0], d.bounds[0,1], d.resolution[0])
    Y = np.linspace(d.bounds[1,0], d.bounds[1,1], d.resolution[1])
    Z = np.linspace(d.bounds[2,0], d.bounds[2,1], d.resolution[2])
    for i in range(d.nlayers):
        #layer = i
        count_arr = np.zeros((len(Y), len(X), len(Z)))
        for frame in range(d.samples):
            z_value_array = d.boundaries[layer][frame]
            #C = pl.cm.jet(i/float(d.nlayers))
            #C = np.array([[[C[0], C[1], C[2]]]])
            #m = vv.surf(X, Y, z_value_array, C)
            #m = vv.surf(X, Y, Z, C)

            voxel_idx_arr = [[np.where(z_val <= Z)[0][0] for z_val in x_array] for x_array in z_value_array]
            #voxel_arr = [[Z[np.where(z_val <= Z)[0][0]] for z_val in x_array] for x_array in z_value_array]
            #m = vv.surf(X, Y, voxel_arr, C, axes = a1)
            for idx_y, y in enumerate(Y):
                for idx_x, x in enumerate(X):
                    idx_z = voxel_idx_arr[idx_y][idx_x]
                    count_arr[idx_y, idx_x, idx_z] += 1
        prob_arr = count_arr/d.samples + prob_arr
        break
prob_arr = prob_arr/n_max
#prob_arr = prob_arr/d.nlayers

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
x, y, z = np.indices((8, 8, 8))

# draw cuboids in the top left and bottom right corners, and a link between them
cube1 = (x < 3) & (y < 3) & (z < 3)
cube2 = (x >= 5) & (y >= 5) & (z >= 5)
link = abs(x - y) + abs(y - z) + abs(z - x) <= 2

# combine the objects into a single boolean array
voxels = cube1 | cube2 | link

In [ ]:
np.shape(voxels)

In [ ]:
np.shape(prob_arr)

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.voxels(prob_arr)
plt.show()

In [ ]:
x, y, z = np.meshgrid(X, Y, Z)

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.voxels(r, g, b, sphere,
          facecolors=colors,
          edgecolors=np.clip(2*colors - 0.5, 0, 1),  # brighter
          linewidth=0.5)
ax.set(xlabel='r', ylabel='g', zlabel='b')

plt.show()

In [ ]:
def midpoints(x):
    sl = ()
    for i in range(x.ndim):
        x = (x[sl + np.index_exp[:-1]] + x[sl + np.index_exp[1:]]) / 2.0
        sl += np.index_exp[:]
    return x

# prepare some coordinates, and attach rgb values to each
r, g, b = np.indices((17, 17, 17)) / 16.0
rc = midpoints(r)
gc = midpoints(g)
bc = midpoints(b)

# define a sphere about [0.5, 0.5, 0.5]
sphere = (rc - 0.5)**2 + (gc - 0.5)**2 + (bc - 0.5)**2 < 0.5**2

# combine the color components
colors = np.zeros(sphere.shape + (3,))
colors[..., 0] = rc
colors[..., 1] = gc
colors[..., 2] = bc

# and plot everything
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.voxels(r, g, b, sphere,
          facecolors=colors,
          edgecolors=np.clip(2*colors - 0.5, 0, 1),  # brighter
          linewidth=0.5)
ax.set(xlabel='r', ylabel='g', zlabel='b')

plt.show()

In [ ]:
app = vv.use()

In [ ]:
vv.xlabel('Eastings (m)')
vv.ylabel('Northings (m)')
vv.zlabel('Depth (m)')
a2 = vv.gca()
a2.camera.fov = 70
#a2.daspect = 1, 1, -1
t = vv.volshow(
    prob_arr,
    cm=vv.CM_JET,
    #renderStyle = 'iso'
)
vv.colorbar()
#vv.ColormapEditor(a2)
t.Draw()

In [ ]:
app.Run()

In [ ]:
vv.screenshot('layer{}.png'.format(layer), sf=10, bg='w')

In [ ]:
import mayavi.mlab as mlab
mlab.init_notebook()

In [ ]:
np.mgrid

In [ ]:
x, y, z = np.meshgrid(X, Y, Z)

In [ ]:
pts = mlab.points3d(x, y, z, prob_arr, scale_factor=.5,transparent=True)

In [ ]:
pts

In [ ]:
%gui qt

In [ ]:
mlab.test_contour3d()

In [ ]:
from mayavi import mlab
mlab.init_notebook()
#mlab.test_contour3d()

In [ ]:
mlab.show()

In [ ]:
import numpy as np

In [ ]:
N = 20
x, y, z = np.mgrid[-5:5:20j, -5:5:20j, -5:5:20j]
val = np.random.random(z.shape)

In [ ]:
np.shape(X)

In [ ]:
prob_arr[np.where(prob_arr)] = 0.5

In [ ]:
np.unique(prob_arr)

In [ ]:
pl.cm.jet(10)

In [ ]:
d.properties[propIdx][0]

# Tests

In [ ]:
app = vv.use()
#frame = 0
#i = 2

In [ ]:
X = np.linspace(d.bounds[0,0], d.bounds[0,1], d.resolution[0])
Y = np.linspace(d.bounds[1,0], d.bounds[1,1], d.resolution[1])
Z = np.linspace(d.bounds[2,0], d.bounds[2,1], d.resolution[2])
#Z = np.linspace(d.bounds[2,0], d.bounds[2,1], 15)

In [ ]:
x_grid, y_grid, z_grid = np.meshgrid(X, Y, Z)

In [ ]:
prob_arr = 0
for i in range(d.nlayers):
    count_arr = np.zeros((len(Y), len(X), len(Z)))
    for frame in range(d.samples):
        z_value_array = d.boundaries[i][frame]
        #C = pl.cm.jet(i/float(d.nlayers))
        #C = np.array([[[C[0], C[1], C[2]]]])
        #m = vv.surf(X, Y, z_value_array, C)
        #m = vv.surf(X, Y, Z, C)

        voxel_idx_arr = [[np.where(z_val <= Z)[0][0] for z_val in x_array] for x_array in z_value_array]
        #voxel_arr = [[Z[np.where(z_val <= Z)[0][0]] for z_val in x_array] for x_array in z_value_array]
        #m = vv.surf(X, Y, voxel_arr, C, axes = a1)
        for idx_y, y in enumerate(Y):
            for idx_x, x in enumerate(X):
                idx_z = voxel_idx_arr[idx_y][idx_x]
                count_arr[idx_y, idx_x, idx_z] += 1
    prob_arr = count_arr/d.samples + prob_arr
    break

In [ ]:
vv.xlabel('Eastings (m)')
vv.ylabel('Northings (m)')
vv.zlabel('Depth (m)')
a2 = vv.gca()
a2.camera.fov = 70
#a2.daspect = 1, 1, -1
t = vv.volshow(prob_arr.T*100,cm=vv.CM_JET)
#t = vv.surf(X, Y, prob_arr, cm=vv.CM_JET)


In [ ]:
app.Run()

In [ ]:
np.shape(prob_arr)

In [ ]:
prob_arr[0,0,:]

In [ ]:
np.shape(d.boundaries[i])

In [ ]:
Z

In [ ]:
sample_idx = 0
for sample in d.boundaries[i]:
    val = sample[0][0]
    print(val)
    idx = np.where(val <= Z)[0][0]
    print(idx)
    print()

In [ ]:
np.shape(d.boundaries[i])

In [ ]:
np.shape(z_value_array[0])

In [ ]:
np.shape(voxel_idx_arr)

In [ ]:
prob_arr

In [ ]:
f = vv.figure(1)
vv.xlabel('Eastings (m)')
vv.ylabel('Northings(m)')
vv.zlabel('Depth (m)')
a1 = vv.gca()
a1.camera.fov = 70
a1.daspect = 1, 1, -1
for i in range(d.nlayers):
    C = pl.cm.jet(i/float(d.nlayers))
    C = np.array([[[C[0], C[1], C[2]]]])
    #m = vv.surf(X, Y, d.boundaries[i][frame], C)
    z_value_array = d.boundaries[i][frame]
    voxel_arr = [[Z[np.where(z_val <= Z)[0][0]] for z_val in x_array] for x_array in z_value_array]
    new_mesh = np.meshgrid(X, Y, voxel_arr)
    #m = vv.surf(X, Y, voxel_arr, C, axes = a1)
    break


In [ ]:
new_mesh

In [ ]:
# Set labels
vv.xlabel('Eastings (m)')
vv.ylabel('Northings (m)')
vv.zlabel('Depth (m)')
a2 = vv.gca()
a2.camera.fov = 70
a2.daspect = 1, 1, -1
vol = d.properties[propIdx][frame]
t = vv.volshow(vol,cm=vv.CM_JET, renderStyle='ray')

In [ ]:
count_arr = np.zeros((len(Y), len(X), len(Z)))
f = vv.figure(1)
X = np.linspace(d.bounds[0,0], d.bounds[0,1], d.resolution[0])
Y = np.linspace(d.bounds[1,0], d.bounds[1,1], d.resolution[1])
Z = np.linspace(d.bounds[2,0], d.bounds[2,1], d.resolution[2])
vv.xlabel('Eastings (m)')
vv.ylabel('Northings(m)')
vv.zlabel('Depth (m)')
a1 = vv.gca()
a1.camera.fov = 70
a1.daspect = 1, 1, -1
i = 0
#for i in range(d.nlayers):
for frame in range(d.samples):
    C = pl.cm.jet(i/float(d.nlayers))
    C = np.array([[[C[0], C[1], C[2]]]])
    #m = vv.surf(X, Y, d.boundaries[i][frame], C)
    z_value_array = d.boundaries[i][frame]
    voxel_idx_arr = [[np.where(z_val <= Z)[0][0] for z_val in x_array] for x_array in z_value_array]
    voxel_arr = [[Z[np.where(z_val <= Z)[0][0]] for z_val in x_array] for x_array in z_value_array]
    #m = vv.surf(X, Y, voxel_arr, C, axes = a1)
    for idx_y, y in enumerate(Y):
        for idx_x, x in enumerate(X):
            idx_z = voxel_idx_arr[idx_y][idx_x]
            count_arr[idx_y, idx_y, idx_z] += 1

In [ ]:
prob_arr = count_arr/d.samples

In [ ]:
# Set labels
vv.xlabel('Eastings (m)')
vv.ylabel('Northings (m)')
vv.zlabel('Depth (m)')
a2 = vv.gca()
a2.camera.fov = 70
a2.daspect = 1, 1, -1
vol = d.properties[propIdx][frame]
t = vv.volshow(prob_arr,cm=vv.CM_JET, renderStyle='ray')

In [ ]:
voxel_idx_arr[0][0]

In [ ]:
np.shape(X)

In [ ]:
np.shape(voxel_idx_arr)

In [ ]:
np.meshgrid()

In [ ]:
x_grid, y_grid, z_grid = np.meshgrid(X, Y, Z)

In [ ]:
np.shape(x_grid)

In [ ]:
x_grid

In [ ]:
aaa = m.GetAxes()

In [ ]:
Z

In [ ]:
m2 = vv.grid(X, Y, d.boundaries[i][frame], C)

In [ ]:
m2.GetFigure

In [ ]:
a1.GetLimits()

In [ ]:
f = vv.figure(1)
X = np.linspace(d.bounds[0,0], d.bounds[0,1], d.resolution[0])
Y = np.linspace(d.bounds[1,0], d.bounds[1,1], d.resolution[1])
Z = np.linspace(d.bounds[2,0], d.bounds[2,1], d.resolution[2])
vv.xlabel('Eastings (m)')
vv.ylabel('Northings(m)')
vv.zlabel('Depth (m)')
a1 = vv.gca()
a1.camera.fov = 70
a1.daspect = 1, 1, -1

In [ ]:
a1.GetLimits()

In [ ]:
f1 = vv.gcf()

In [ ]:
t1 = vv.imshow(m, axes=a1)

In [ ]:
aaa = m.GetAxes()

In [ ]:
aaa.GetLimits()

In [ ]:
aaa.GetView()

In [ ]:
az = aaa.axis

In [ ]:
fff = az.GetFigure()

In [ ]:
Z

In [ ]:
np.shape(d.boundaries[i][frame])

In [ ]:
# X and Y are the points of the grid
# i.e. for every point at Y make all points in X
# d.boundaries[i][frame] is the volume at each point in the grid

In [ ]:
Z

In [ ]:
i = 1
z_value_array = d.boundaries[i][frame]
voxel_arr = [[Z[np.where(z_val <= Z)[0][0]] for z_val in x_array] for x_array in z_value_array]
#bin_idx = (z_value_array[0, 0] <= Z)
#voxel_idx = np.where(bin_idx)[0][0]

In [ ]:
voxel_idx_arr

In [ ]:
z_value_array<= Z

In [ ]:
i = 1
vol_point = d.boundaries[i][frame]
print(np.shape(vol_point))
print(vol_point)

In [ ]:
vol_point[0, 0]

In [ ]:
vol_point[0, 0] < Z

In [ ]:
idx_arr = (vol_point[0, 0] <= Z)
np.where(idx_arr)[0][0]

In [ ]:
idx_arr

In [ ]:
vol_point[0, 0] > Z

In [ ]:
for v in vol_point:
    

In [ ]:
np.shape(X)

In [ ]:
np.shape(Y)

In [ ]:
a_test = f.children[0]

In [ ]:
temp = a_test.GetAxes()

In [ ]:
temp.GetLimits()

In [ ]:
temp.